Comparing with only Pumping Model

In [ ]:
def gen_only_objective(g):
    return -np.sum(prices * g)  # Only generation earns revenue

def compute_storage_gen_only(g):
    S = [initial_storage]
    for t in range(T):
        S.append(S[-1] - gen_efficiency * g[t] + inflows[t])
    return np.array(S)

def run_gen_only_model():
    # Constraints for gen-only case
    gen_only_constraints = []
    for t in range(1, T + 1):
        gen_only_constraints.append({
            'type': 'ineq',
            'fun': lambda g, t=t: compute_storage_gen_only(g)[t] - S_min
        })
        gen_only_constraints.append({
            'type': 'ineq',
            'fun': lambda g, t=t: reservoir_capacity - compute_storage_gen_only(g)[t]
        })
    gen_only_constraints.append({
        'type': 'eq',
        'fun': lambda g: compute_storage_gen_only(g)[-1] - initial_storage
    })

    bounds_gen_only = [(0, gen_max)] * T
    g0 = np.zeros(T)

    result = minimize(gen_only_objective, g0, method='SLSQP', bounds=bounds_gen_only, constraints=gen_only_constraints)
    
    if result.success:
        g_opt = result.x
        S_opt = compute_storage_gen_only(g_opt)
        total_profit = -gen_only_objective(g_opt)
        print("Gen-Only Model:")
        print("  Total Daily Profit: €", round(total_profit, 2))
        return g_opt, S_opt, total_profit
    else:
        print("Gen-Only Model: Optimization did not converge.")
        return None, None, None
g_gen_only, S_gen_only, profit_gen_only = run_gen_only_model()